<a href="https://colab.research.google.com/github/tomaszwk/NLP/blob/main/Desafio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [30]:
import os
import requests

# URL del libro en formato txt desde el Proyecto Gutenberg
url = 'https://www.gutenberg.org/files/2701/2701-0.txt'  # Ejemplo con "Moby Dick" de Herman Melville

# Nombre del archivo donde se guardará el libro
filename = 'moby_dick.txt'

# Verificar si el archivo ya existe
if not os.path.exists(filename):
    print("Descargando el libro desde el Proyecto Gutenberg...")
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f'El libro se ha descargado y guardado como {filename}')
    else:
        print("Hubo un problema al descargar el libro.")
else:
    print(f"El archivo '{filename}' ya existe.")



Descargando el libro desde el Proyecto Gutenberg...
El libro se ha descargado y guardado como moby_dick.txt


In [3]:
# Posibles bandas
os.listdir("./songs_dataset/")

['blink-182.txt',
 'paul-simon.txt',
 'patti-smith.txt',
 'dr-seuss.txt',
 'johnny-cash.txt',
 'lil-wayne.txt',
 'michael-jackson.txt',
 'bob-marley.txt',
 'al-green.txt',
 'nirvana.txt',
 'adele.txt',
 'dickinson.txt',
 'prince.txt',
 'nursery_rhymes.txt',
 'beatles.txt',
 'notorious-big.txt',
 'missy-elliott.txt',
 'cake.txt',
 'kanye-west.txt',
 'ludacris.txt',
 'nicki-minaj.txt',
 'disney.txt',
 'eminem.txt',
 'lin-manuel-miranda.txt',
 'dolly-parton.txt',
 'joni-mitchell.txt',
 'notorious_big.txt',
 'nickelback.txt',
 'lorde.txt',
 'leonard-cohen.txt',
 'amy-winehouse.txt',
 'drake.txt',
 'kanye.txt',
 'Kanye_West.txt',
 'janisjoplin.txt',
 'radiohead.txt',
 'bruno-mars.txt',
 'bruce-springsteen.txt',
 'bob-dylan.txt',
 'rihanna.txt',
 'britney-spears.txt',
 'lady-gaga.txt',
 'bieber.txt',
 'dj-khaled.txt',
 'r-kelly.txt',
 'Lil_Wayne.txt',
 'alicia-keys.txt',
 'bjork.txt',
 'jimi-hendrix.txt']

In [31]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('moby_dick.txt', sep='/n', header=None)
df.head()

<ipython-input-31-0c7f1ded6659>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,The Project Gutenberg eBook of Moby-Dick; or T...
1,This eBook is for the use of anyone anywhere i...
2,most other parts of the world at no cost and w...
3,"whatsoever. You may copy it, give it away or r..."
4,of the Project Gutenberg License included with...


In [32]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 19229


### 1 - Preprocesamiento

In [6]:
pip install Keras==2.12.0


  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: Keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires keras<2.11,>=2.10.0, but you have keras 2.12.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.10.0 which is incompatible.


In [7]:
pip install tensorflow==2.10.0

  Using cached keras-2.10.0-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.10.0 which is incompatible.


In [33]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [34]:
# Demos un vistazo
sentence_tokens[:2]

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'moby',
  'dick',
  'or',
  'the',
  'whale',
  'by',
  'herman',
  'melville'],
 ['this',
  'ebook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'united',
  'states',
  'and']]

### 2 - Crear los vectores (word2vec)

In [35]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [36]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [37]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [38]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 19229


In [39]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4381


### 3 - Entrenar embeddings

In [40]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1675047.625
Loss after epoch 1: 1121795.875
Loss after epoch 2: 1057600.75
Loss after epoch 3: 1010644.75
Loss after epoch 4: 987850.0
Loss after epoch 5: 977799.5
Loss after epoch 6: 967653.5
Loss after epoch 7: 947658.0
Loss after epoch 8: 923223.0
Loss after epoch 9: 912971.0
Loss after epoch 10: 907722.0
Loss after epoch 11: 896458.0
Loss after epoch 12: 889036.0
Loss after epoch 13: 882084.0
Loss after epoch 14: 875566.0
Loss after epoch 15: 871382.0
Loss after epoch 16: 867185.0
Loss after epoch 17: 836019.0
Loss after epoch 18: 835232.0
Loss after epoch 19: 829638.0


(2826904, 4415420)

### 4 - Ensayar

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["whale"], topn=10)

[('hairs', 0.6063191890716553),
 ('glistening', 0.5978313684463501),
 ('whales', 0.5961450934410095),
 ('whale’s', 0.5960444808006287),
 ('“hast', 0.5859410762786865),
 ('squalls', 0.5779882073402405),
 ('shoal', 0.5764021873474121),
 ('whaler', 0.5711514353752136),
 ('steed', 0.5658363699913025),
 ('hunting', 0.5630220174789429)]

In [43]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('or', -0.09040482342243195),
 ('into', -0.12280155718326569),
 ('than', -0.1497901827096939),
 ('its', -0.1548781543970108),
 ('from', -0.15526516735553741),
 ('no', -0.15613260865211487),
 ('between', -0.15714167058467865),
 ('up', -0.15834391117095947),
 ('broad', -0.16433435678482056),
 ('any', -0.16784261167049408)]

In [44]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('sixty', 0.7609437108039856),
 ('weeks', 0.7402021884918213),
 ('ninety', 0.7294574975967407),
 ('five', 0.7290971875190735),
 ('eighteen', 0.7273372411727905),
 ('thirty', 0.7200717926025391),
 ('twenty', 0.7117601037025452),
 ('score', 0.7077972888946533),
 ('years’', 0.7056097984313965),
 ('fifty', 0.7010278701782227)]

In [45]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('religion', 0.8330126404762268),
 ('murderer', 0.8178383708000183),
 ('hopeless', 0.8125391602516174),
 ('shown', 0.8086636066436768),
 ('sober', 0.8084625601768494)]

In [21]:
# Ensayar con una palabra que no está en el vocabulario:
#w2v_model.wv.most_similar(negative=["diedaa"])

In [23]:
# el método `get_vector` permite obtener los vectores:
vector_love = w2v_model.wv.get_vector("love")
print(vector_love)

[ 0.17736822  0.21460655 -0.05408018  0.06707036  0.13059464 -0.25713933
  0.00711611  0.26743424 -0.03100746  0.259963    0.21179457 -0.08465387
 -0.26272097  0.308386   -0.07934437 -0.13357355  0.2994755   0.18062147
 -0.13111673 -0.11857472  0.03468326  0.1125607   0.08478875  0.08090881
 -0.17622267  0.03122034 -0.08725598  0.23746666 -0.11507358 -0.29119617
 -0.03650665 -0.3322312   0.23262788  0.10366543 -0.18365337  0.08299578
  0.14315338 -0.19761457 -0.00757894  0.1119568  -0.10421463 -0.13083619
 -0.03264419 -0.04687435  0.11330808  0.10231452 -0.16202034 -0.10347028
  0.03659901 -0.08524423 -0.34872082 -0.02154255  0.25938764  0.16540806
  0.0310861   0.09505633  0.24452917 -0.09859891  0.18298703  0.0645447
  0.016967   -0.3259168  -0.04774074 -0.05418535 -0.04338397 -0.09137302
  0.00889874  0.11636022 -0.25662988  0.05426472  0.08915784  0.04541444
  0.20835714 -0.19101828  0.32210803  0.19337991  0.09080223 -0.00362429
 -0.18664253 -0.07053486 -0.1213743  -0.03477469 -0.

In [24]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_love)

[('love', 1.0),
 ('babe', 0.9132986068725586),
 ('someone', 0.8914992809295654),
 ('nothing', 0.8811879754066467),
 ('need', 0.8758756518363953),
 ("didn't", 0.8689683079719543),
 ("there's", 0.8557919859886169),
 ('you', 0.8447725772857666),
 ('feed', 0.8422104716300964),
 ('somebody', 0.8385539054870605)]

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

### 5 - Visualizar agrupación de vectores

In [46]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)


    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [47]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [27]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [28]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.